# Bitacora para el manejo de secuencias provenientes de un [microbioma de la piel de larvas de bacalao negro secuenciado con Illumina MiSeq.](https://www.ncbi.nlm.nih.gov/sra/?term=SRX6435585)
referencia:
[Pierce et al.](https://www.frontiersin.org/articles/10.3389/fmars.2019.00203/full)

### Revise la página del [NCBI](https://www.ncbi.nlm.nih.gov/sra/?term=SRX6435585) y describa la muestra en la siguiente celda utilizando markdown

## Se analizarán con *Blastn* y la base de datos de 16SMicrobial

In [ ]:
import os
from pandas import Series, DataFrame
import pandas as pd
from Bio import SeqIO, AlignIO, SeqRecord
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import matplotlib.pyplot as plt 

In [ ]:
cd ~/Desktop/data/fasta/

In [ ]:
ls

In [ ]:
!head Pierce287.3.14L13.S.S43046070.fasta

### Crear el directorio "img"

In [ ]:
os.makedirs('img',exist_ok=True)

In [ ]:
!grep ">" Pierce287.3.14L13.S.S43046070.fasta |wc -l

# Se analizarán con blastn los contigs obtenidos a la base de datos *16SMicrobial*

Verifique la localización de la base de datos, en este caso se encuentra en `/home/database/16SMicrobial` o corrija si es necesario

In [ ]:
%%bash
#export BLASTDB=~/DATA/nt/
export BLASTDB=/home/database/16SMicrobial
date  
time blastn -query Pierce287.3.14L13.S.S43046070.fasta \
-db /home/database/16SMicrobial/16SMicrobial \
-out pierce16s.tsv \
-evalue 1E-6 \
-max_target_seqs 1 \
-num_threads 18 \
-outfmt "6 std sskingdoms stitle staxids sscinames scomnames sblastnames sstrand" 
date

In [ ]:
%%bash
#export BLASTDB=~/DATA/nt/
export BLASTDB=~/Desktop/bigdata/16SMicrobial/
date  
time blastn -query Pierce287.3.14L13.S.S43046070.fasta \
-db ~/Desktop/bigdata/16SMicrobial/16SMicrobial \
-out pierce16s.tsv \
-evalue 1E-6 \
-max_target_seqs 1 \
-num_threads 18 \
-outfmt "6 std sskingdoms stitle staxids sscinames scomnames sblastnames sstrand" 
date

In [ ]:
ls

In [ ]:
!head -2 pierce16s.tsv

In [ ]:
encabezado =("qseqid", "sseqid", "pident", "length", "mismatch", "gapopen","qstart", 
             "qend", "sstart","send", "evalue", "bitscore", "sskingdoms", "stitle", 
             "staxids", "sscinames", "scomnames", "sblastnames","sstrand")

In [ ]:
ftab=pd.read_csv("pierce16s.tsv", header=None, sep = "\t" , names= encabezado)
ftab.head()

In [ ]:
len(ftab)

In [ ]:
ftab1= ftab.groupby("sskingdoms")["qseqid"].count()
ftab1 = DataFrame(ftab1)
ftab1

# Cuántos contigs no son bacterianos?

## Guadardo la base de datos en formato csv

In [ ]:
ftab.to_csv("pierce_blastn16s.csv", header=True, index= None)

### en caso de haber el hecho el análisis previo y querer recuperar el archivo anterior 

In [ ]:
ftab= pd.read_csv("pierce_blastn16s.csv")
ftab.head(2)

# Hay algún contig con más de una asignación (duplicados)?

In [ ]:
ftab1= ftab.groupby("qseqid")["qseqid"].count()
ftab1 = DataFrame(ftab1)
ftab1

# Es necesario eliminar duplicados?

In [ ]:
ftab1 =ftab.drop_duplicates(subset = 'qseqid', keep='first', inplace = False)
ftab1

# Cuántos grupos hay a parte del que pertenece la especie analizada?

In [ ]:
ftab2= ftab1.groupby("sblastnames")["qseqid"].count()
ftab2 = DataFrame(ftab2)
ftab2

In [ ]:
ftab2= ftab1.groupby("sscinames")["qseqid"].count()
ftab2.sort_values(axis = 0, ascending=False, inplace=True)
ftab2

# Procedimiento para simplificar tabla y graficar las 10 primeras categorías y el resto ponerlas en "otras"

In [ ]:
linea10=ftab2[:10]
linea11=ftab2[10:]
#linea10
otro=sum (linea11)
#otro
otros = pd.DataFrame({0:otro}, index=["Other"])
otros
linea10=linea10.append(otros)
linea10

In [ ]:
linea10.plot(kind='barh', figsize= (8,6))
plt.axis([-1, int(max(linea10[0])*1.05), -1, 10.8], label=None)
plt.legend().set_visible(False)
plt.xlabel("Frecuencia")
plt.ylabel("Especies")
plt.title("Especies con resultado de blastn")
plt.show()